In [5]:
import requests
import pandas as pd

def make_fs_dataframe(firm_code):
    fs_url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701&gicode=' + firm_code
    fs_page = requests.get(fs_url)
    fs_tables = pd.read_html(fs_page.text)

    temp_df = fs_tables[0]
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df = temp_df[temp_df.columns[:4]]
    temp_df = temp_df.loc[['매출액','영업이익','당기순이익']]

    temp_df2 = fs_tables[2]
    temp_df2 = temp_df2.set_index(temp_df2.columns[0])
    temp_df2 = temp_df2.loc[['자산','부채','자본']]

    temp_df3 = fs_tables[4]
    temp_df3 = temp_df3.set_index(temp_df3.columns[0])
    temp_df3 = temp_df3.loc[['영업활동으로인한현금흐름']]
    
    fs_df = pd.concat([temp_df, temp_df2, temp_df3])
    
    return fs_df

In [6]:
def change_df(firm_code, dataframe):
    for num, col in enumerate(dataframe.columns):
        temp_df = pd.DataFrame({firm_code:dataframe[col]})
        temp_df = temp_df.T
        temp_df.columns = [[col]*len(dataframe), temp_df.columns]

        if num == 0:
            total_df = temp_df
        else:
            total_df = pd.merge(total_df, temp_df, how='outer', left_index=True, right_index=True)
            
    return total_df

In [7]:
firm_code_list = ['A035420']

for num, code in enumerate(firm_code_list):
    fs_df = make_fs_dataframe(code)
    fs_df_changed = change_df(code, fs_df)
    
    if num == 0:
        total_fs = fs_df_changed
    else:
        total_fs = pd.concat([total_fs, fs_df_changed])

In [9]:
total_fs

2016/12                                                           \
IFRS(연결)      매출액     영업이익   당기순이익       자산       부채       자본 영업활동으로인한현금흐름   
A035420   40226.0  11020.0  7591.0  63706.0  22410.0  41296.0      11640.0   

          2017/12                   ...  2018/12                        \
IFRS(연결)      매출액     영업이익   당기순이익  ...       부채       자본 영업활동으로인한현금흐름   
A035420   46785.0  11792.0  7701.0  ...  39321.0  59491.0       9735.0   

          2019/12                                                           
IFRS(연결)      매출액    영업이익   당기순이익        자산       부채       자본 영업활동으로인한현금흐름  
A035420   65934.0  7101.0  3968.0  122995.0  57956.0  65039.0      13568.0  

[1 rows x 28 columns]